# Learnplatform Covid19 Impact on Digital Learning

## This Notebook is deivided into Two main parts

## 1. Data Pre-processing
## 2. Exploratory Data Analysis (EDA)

## Data Reading and Pre-processing

### Importing Required packages and Libraries

In [ ]:
# Importing necessary packages